In [103]:
import os
import psycopg2
from dotenv import load_dotenv
load_dotenv()

True

In [104]:
dbname = os.getenv('POSTGRES_DB')
user = os.getenv('POSTGRES_USER')
password = os.getenv('POSTGRES_PASSWORD')
db_host = os.getenv('DB_HOST')
db_post = os.getenv('DB_PORT')
try:
    # пытаемся подключиться к базе данных
    conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=db_host, port=db_post)
except:
    # в случае сбоя подключения будет выведено сообщение в STDOUT
    print('Can`t establish connection to database')

In [105]:
cursor = conn.cursor()
conn.autocommit = True

# 1. Создайте таблицу movies с полями movies_type, director, year_of_issue, length_in_minutes, rate.

In [106]:
sql = '''
CREATE TABLE IF NOT EXISTS movies(
id bigint not null,
movies_type int not null,
director VARCHAR(50),
year_of_issue DATE NOT NULL,
length_in_minutes numeric NOT NULL,
rate int not null
)'''
cursor.execute(sql)
conn.commit()

In [107]:
cursor.execute("SELECT * from movies;")
cursor.fetchall()

[]

# 2 Сделайте таблицы для горизонтального партицирования по году выпуска (до 1990, 1990 -2000, 2000- 2010, 2010-2020, после 2020).

In [108]:
cursor.execute("CREATE TABLE IF NOT EXISTS year_1990 (CHECK (EXTRACT(YEAR FROM year_of_issue)<1990)) INHERITS (movies);")
cursor.execute("CREATE TABLE IF NOT EXISTS year_1990_2000 (CHECK (EXTRACT(YEAR FROM year_of_issue)>=1990 AND EXTRACT(YEAR FROM year_of_issue)<2000)) INHERITS (movies);")
cursor.execute("CREATE TABLE IF NOT EXISTS year_2000_2010 (CHECK (EXTRACT(YEAR FROM year_of_issue)>=2000 AND EXTRACT(YEAR FROM year_of_issue)<2010)) INHERITS (movies);")
cursor.execute("CREATE TABLE IF NOT EXISTS year_2010_2020 (CHECK (EXTRACT(YEAR FROM year_of_issue)>=2010 AND EXTRACT(YEAR FROM year_of_issue)<2020)) INHERITS (movies);")
cursor.execute("CREATE TABLE IF NOT EXISTS year_2020 (CHECK (EXTRACT(YEAR FROM year_of_issue)>=2020)) INHERITS (movies);")
conn.commit()

In [109]:
cursor.execute("CREATE RULE year_1990_r AS ON INSERT TO movies WHERE (EXTRACT(YEAR FROM year_of_issue)<1990) DO INSTEAD INSERT INTO year_1990 VALUES (NEW.*);")
cursor.execute("CREATE RULE year_1990_2000_r AS ON INSERT TO movies WHERE (EXTRACT(YEAR FROM year_of_issue)>=1990 AND EXTRACT(YEAR FROM year_of_issue)<2000) DO INSTEAD INSERT INTO year_1990_2000 VALUES (NEW.*);")
cursor.execute("CREATE RULE year_2000_2010_r AS ON INSERT TO movies WHERE (EXTRACT(YEAR FROM year_of_issue)>=2000 AND EXTRACT(YEAR FROM year_of_issue)<2010) DO INSTEAD INSERT INTO year_2000_2010 VALUES (NEW.*);")
cursor.execute("CREATE RULE year_2010_2020_r AS ON INSERT TO movies WHERE (EXTRACT(YEAR FROM year_of_issue)>=2010 AND EXTRACT(YEAR FROM year_of_issue)<2020) DO INSTEAD INSERT INTO year_2010_2020 VALUES (NEW.*);")
cursor.execute("CREATE RULE year_2020_r AS ON INSERT TO movies WHERE (EXTRACT(YEAR FROM year_of_issue)>=2020) DO INSTEAD INSERT INTO year_2020 VALUES (NEW.*);")
conn.commit()

# 3  Сделайте таблицы для горизонтального партицирования по длине фильма (до 40 минута, от 40 до 90 минут, от 90 до 130 минут, более 130 минут).

In [110]:
cursor.execute("CREATE TABLE IF NOT EXISTS length_in_minutes_40 (CHECK (length_in_minutes<40)) INHERITS (movies);")
cursor.execute("CREATE TABLE IF NOT EXISTS length_in_minutes_40_90 (CHECK (length_in_minutes>=40 AND length_in_minutes<90)) INHERITS (movies);")
cursor.execute("CREATE TABLE IF NOT EXISTS length_in_minutes_90_40 (CHECK (length_in_minutes>=90 AND length_in_minutes<130)) INHERITS (movies);")
cursor.execute("CREATE TABLE IF NOT EXISTS length_in_minutes_130 (CHECK (length_in_minutes>=130)) INHERITS (movies);")
conn.commit()

In [111]:
cursor.execute("CREATE RULE length_in_minutes_40_r AS ON INSERT TO movies WHERE (length_in_minutes<40) DO INSTEAD INSERT INTO length_in_minutes_40 VALUES (NEW.*);")
cursor.execute("CREATE RULE length_in_minutes_40_90_r AS ON INSERT TO movies WHERE (length_in_minutes>=40 AND length_in_minutes<90) DO INSTEAD INSERT INTO length_in_minutes_40_90 VALUES (NEW.*);")
cursor.execute("CREATE RULE length_in_minutes_90_40_r AS ON INSERT TO movies WHERE (length_in_minutes>=90 AND length_in_minutes<130) DO INSTEAD INSERT INTO length_in_minutes_90_40 VALUES (NEW.*);")
cursor.execute("CREATE RULE length_in_minutes_130_r AS ON INSERT TO movies WHERE (length_in_minutes>=130) DO INSTEAD INSERT INTO length_in_minutes_130 VALUES (NEW.*);")
conn.commit()

# 4. Сделайте таблицы для горизонтального партицирования по рейтингу фильма (ниже 5, от 5 до 8, от 8до 10).

In [112]:
cursor.execute("CREATE TABLE IF NOT EXISTS rate_5(CHECK (rate<5)) INHERITS (movies);")
cursor.execute("CREATE TABLE IF NOT EXISTS rate_5_8(CHECK (rate>=5 AND rate<8)) INHERITS (movies);")
cursor.execute("CREATE TABLE IF NOT EXISTS rate_8_10(CHECK (rate>=5 AND rate<10)) INHERITS (movies);")
conn.commit()

In [113]:
cursor.execute("CREATE RULE rate_5_r AS ON INSERT TO movies WHERE (rate<5) DO INSTEAD INSERT INTO rate_5 VALUES (NEW.*);")
cursor.execute("CREATE RULE rate_5_8_r AS ON INSERT TO movies WHERE (rate>=5 AND rate<8) DO INSTEAD INSERT INTO rate_5_8 VALUES (NEW.*);")
cursor.execute("CREATE RULE rate_8_10_r AS ON INSERT TO movies WHERE (rate>=8 AND rate<10) DO INSTEAD INSERT INTO rate_8_10 VALUES (NEW.*);")
conn.commit()

In [114]:
cursor.execute("INSERT INTO movies (id, movies_type, director, year_of_issue, length_in_minutes, rate) VALUES (1, 1, 'first', '1990-5-6', 90, 2)")
cursor.execute("INSERT INTO movies (id, movies_type, director, year_of_issue, length_in_minutes, rate) VALUES (2, 2, 'second', '1990-1-1', 40, 6)")
cursor.execute("INSERT INTO movies (id, movies_type, director, year_of_issue, length_in_minutes, rate) VALUES (3, 3, 'first', '2020-2-2', 100, 9)")
conn.commit()


In [120]:
cursor.execute("SELECT * FROM pg_catalog.pg_tables ")
cursor.fetchall()

[('pg_catalog', 'pg_statistic', 'postgres', None, True, False, False, False),
 ('pg_catalog', 'pg_type', 'postgres', None, True, False, False, False),
 ('public', 'year_2010_2020', 'alexandr', None, False, False, False, False),
 ('public', 'year_1990', 'alexandr', None, False, False, False, False),
 ('public', 'year_1990_2000', 'alexandr', None, False, False, False, False),
 ('public', 'year_2020', 'alexandr', None, False, False, False, False),
 ('public', 'year_2000_2010', 'alexandr', None, False, False, False, False),
 ('public', 'movies', 'alexandr', None, False, True, False, False),
 ('public',
  'length_in_minutes_40',
  'alexandr',
  None,
  False,
  False,
  False,
  False),
 ('public',
  'length_in_minutes_130',
  'alexandr',
  None,
  False,
  False,
  False,
  False),
 ('public',
  'length_in_minutes_40_90',
  'alexandr',
  None,
  False,
  False,
  False,
  False),
 ('public', 'rate_5', 'alexandr', None, False, False, False, False),
 ('public',
  'length_in_minutes_90_40',
 

In [122]:
cursor.execute("SELECT * FROM rate_5")
cursor.fetchall()

[(1, 1, 'first', datetime.date(1990, 5, 6), Decimal('90'), 2)]

In [123]:
cursor.execute("SELECT * FROM rate_5_8")
cursor.fetchall()

[(2, 2, 'second', datetime.date(1990, 1, 1), Decimal('40'), 6)]

In [124]:
cursor.execute("SELECT * FROM rate_8_10")
cursor.fetchall()

[(3, 3, 'first', datetime.date(2020, 2, 2), Decimal('100'), 9)]

In [125]:
cursor.execute("INSERT INTO movies (id, movies_type, director, year_of_issue, length_in_minutes, rate) VALUES (1, 1, 'first', '1990-5-6', 90, 50)")
conn.commit()

In [126]:
cursor.execute("SELECT * FROM rate_8_10")
cursor.fetchall()

[(3, 3, 'first', datetime.date(2020, 2, 2), Decimal('100'), 9)]

In [128]:
cursor.execute("SELECT * FROM movies WHERE rate=50")
cursor.fetchall()

[(1, 1, 'first', datetime.date(1990, 5, 6), Decimal('90'), 50),
 (1, 1, 'first', datetime.date(1990, 5, 6), Decimal('90'), 50)]

In [129]:
cursor.close()
conn.close()